# Image Denoising

We'll demonstrate the use of a typical encoder–decoder by denoising a very noisy brightfield image of a single particle.

## Simulate a Clean and Noisy Particle

We use `Deeptrack 2.1` to simulate a spherical particle.

In [ ]:
import deeptrack as dt
import numpy as np

particle = dt.Sphere(
    position=np.array([0.5, 0.5]) * 64,
    position_unit="pixel",
    # z=0,  ### BM: line can be removed
    radius=500 * dt.units.nm,
    refractive_index=1.45 + 0.02j,
)

We define the optical sysetm to image the particle.

In [ ]:
brightfield_microscope = dt.Brightfield(
    wavelength=500 * dt.units.nm,
    NA=1,
    resolution=1 * dt.units.um,
    magnification=10,
    refractive_index_medium=1.33,
    output_region=(0, 0, 64, 64),
    upsample=2,  ### Carlo: present in original version, seems to make a big difference in generalization
)

We create the sample by applying the optical system to the particle.

In [ ]:
illuminated_sample = brightfield_microscope(particle)

We create a workflow to simulate _clean_ particle images. We also corrupt them with some Poisson noise to have their _noisy_ version. We join the two workflows into a single simulation pipeline.

In [ ]:
import torch

clean_particle = (  ### Carlo: as in the book chapter
    illuminated_sample
    >> dt.NormalizeMinMax()
    >> dt.MoveAxis(2, 0)
    >> dt.pytorch.ToTensor(dtype=torch.float)
)

# pipeline = (
#     (illuminated_sample >> noise & illuminated_sample)
#     >> dt.NormalizeMinMax()
#     >> dt.MoveAxis(2, 0)
#     >> dt.pytorch.ToTensor(dtype=torch.float)
# )

In [ ]:
noise = dt.Poisson(snr=lambda: 2.0 + np.random.rand())  ### Carlo: *1 not needed

noisy_particle = (
    illuminated_sample
    >> noise
    >> dt.NormalizeMinMax()
    >> dt.MoveAxis(2, 0)
    >> dt.pytorch.ToTensor(dtype=torch.float)
)

In [ ]:
pipeline = (noisy_particle) & (clean_particle)

We plot an example of the generated clean and noisy particle. 

For this, we write a simple function to plot the images and add it to `for_denoising.py`:

```python
import matplotlib.pyplot as plt


def plot_image(title, image):
    """Plot a grayscale image with a title."""
    
    plt.imshow(image, cmap="gray")
    plt.title(title, fontsize=24)
    plt.axis("off")
    plt.show()
```

In [ ]:
from for_denoising import plot_image

for i in range(5):
    input, target = pipeline.update().resolve()

    plot_image(f"Input Image {i}", input.permute(1, 2, 0))
    plot_image(f"Target Image {i}", target.permute(1, 2, 0))

## Create a Dataset

We define a dataset class that uses the simulation pipeline to generate data. When calling the class, we establish the size of dataset but also introduce the possibility to replace some of the initial data on the fly. In this way we can increase the amount of data that the network sees during the whole training while we keep the dataset size fixed at each epoch, thus establishing a trade off between simulation/training time and generalization ability.

For this we'll create a class and add it to `for_denoising.py`:

```python
class SimulatedDataset(Dataset):
    """Simulated dataset generating pairs of noisy and clean images."""
    
    def __init__(self, pipeline, buffer_size, replace=0):
        """Initialize the dataset."""
        
        self.buffer_size = buffer_size
        self.pipeline = pipeline
        self.replace = replace
        self.images = [pipeline.update().resolve() for _ in range(buffer_size)]

    def __len__(self):
        """Return the size of the dataset buffer."""
        
        return self.buffer_size

    def __getitem__(self, idx):
        """Retrieve a noisy-clean image pair from the dataset."""
        
        if np.random.rand() < self.replace:
            self.images[idx] = self.pipeline.update().resolve()
            
        image_pair = self.images[idx]
        noisy_image, clean_image = image_pair[0], image_pair[1]
        
        return noisy_image, clean_image
```

We create the datasets and the data loaders. These will be used both for the training and for the testing, as each new image is never seen before.

In [ ]:
from torch.utils.data import DataLoader
from for_denoising import SimulatedDataset

dataset = SimulatedDataset(
    pipeline, buffer_size=256, replace=0.1
)  ### Carlo: with 0.1, training more stable
loader = DataLoader(dataset, batch_size=8, shuffle=True)

## Define and Train the Encoder-decoder

We define the encoder-decoder architecture.

In [ ]:
import deeplay as dl

# from torch.nn import Sigmoid ### Carlo: not necessary


ed = dl.ConvolutionalEncoderDecoder2d(
    in_channels=1,
    encoder_channels=[16, 16],
    decoder_channels=[16, 16],
    out_channels=1,
    # out_activation=Sigmoid,  ### Carlo: already by default
)

print(ed)

We define the loss and the optimizer...

In [ ]:
from torch.nn import L1Loss

regressor_template = dl.Regressor(model=ed, loss=L1Loss(), optimizer=dl.Adam())
ed_regressor = regressor_template.create()
print(ed_regressor)

... and start the training.

In [ ]:
ed_trainer = dl.Trainer(max_epochs=100, accelerator="auto")

ed_trainer.fit(ed_regressor, loader)

We visualize the encoder-decoder predictions.

In [ ]:
for i in range(5):
    input, target = pipeline.update().resolve()
    predicted = ed_regressor(input.unsqueeze(0)).detach()

    plot_image(f"Input Image {i}", input[0, :, :])
    plot_image(f"Target Image {i}", target[0, :, :])
    plot_image(f"Predicted Image {i}", predicted[0, 0, :, :])

## Check the Absence of Mode Collapse

The training images are very similar. It can be that the encoder-decoder just learn to provide a centered particle, no matter what is the input (mode collapse). Let's check it out by making a prediction out of noise. 

In [ ]:
import torch

blank = brightfield_microscope(particle ^ 0)

blank_pipeline = (
    blank
    >> noise
    >> dt.NormalizeMinMax()
    >> dt.MoveAxis(2, 0)
    >> dt.pytorch.ToTensor(dtype=torch.float)
)

for i in range(5):
    blank_image = blank_pipeline.update().resolve()
    blank_predicted = ed_regressor(blank_image.unsqueeze(0)).detach()

    plot_image(f"Input Image {i}", blank_image[0, :, :])
    plot_image(
        f"Predicted Image {i}", np.square(blank_predicted[0, 0, :, :])
    )  ### BM: maybe normalize the visualization so that the images are comparable

## Check Generalization Capabilities

Let's see if the encoder-decoder is able to generalize its predictions to particles located off center and with different size.

In [ ]:
diverse_particle = dt.Sphere(
    position=lambda: np.array([0.2, 0.2] + np.random.rand(2) * 0.6) * 64,
    # z=0,  ### Carlo: not necessary
    position_unit="pixel",
    radius=lambda: 500 * dt.units.nm * (1 + np.random.rand()),
    ### Carlo: changed for consistency
    refractive_index=1.45 + 0.02j,
)

diverse_illuminated_sample = brightfield_microscope(diverse_particle)

# ### Carlo: changed for consistency
# diverse_pipeline = (
#     (diverse_illuminated_sample >> noise & diverse_illuminated_sample)
#     >> dt.NormalizeMinMax()
#     >> dt.MoveAxis(2, 0)
#     >> dt.pytorch.ToTensor(dtype=torch.float)
# )

### Carlo: changed back as in he book
diverse_clean_particle = (
    diverse_illuminated_sample
    >> dt.NormalizeMinMax()
    >> dt.MoveAxis(2, 0)
    >> dt.pytorch.ToTensor(dtype=torch.float)
)

diverse_noisy_particle = (
    diverse_illuminated_sample
    >> noise
    >> dt.NormalizeMinMax()
    >> dt.MoveAxis(2, 0)
    >> dt.pytorch.ToTensor(dtype=torch.float)
)

diverse_pipeline = (diverse_noisy_particle) & (diverse_clean_particle)

We check the predictions on this more challenging test set.

In [ ]:
for i in range(5):
    diverse_input, diverse_target = diverse_pipeline.update().resolve()
    diverse_predicted = ed_regressor(diverse_input.unsqueeze(0)).detach()

    plot_image(f"Input Image {i}", diverse_input[0, :, :])
    plot_image(f"Target Image {i}", diverse_target[0, :, :])
    plot_image(f"Predicted Image {i}", diverse_predicted[0, 0, :, :])

We can train on the new dataset.

In [ ]:
diverse_dataset = SimulatedDataset(diverse_pipeline, buffer_size=256, replace=0.1)
diverse_loader = DataLoader(diverse_dataset, batch_size=8, shuffle=True)
diverse_ed_regressor = regressor_template.create()
diverse_ed_trainer = dl.Trainer(max_epochs=100, accelerator="auto")
diverse_ed_trainer.fit(diverse_ed_regressor, diverse_loader)

In [ ]:
for i in range(5):
    diverse_input, diverse_target = diverse_pipeline.update().resolve()
    diverse_predicted = diverse_ed_regressor(diverse_input.unsqueeze(0)).detach()

    plot_image(f"Input Image {i}", diverse_input[0, :, :])
    plot_image(f"Target Image {i}", diverse_target[0, :, :])
    plot_image(f"Predicted Image {i}", diverse_predicted[0, 0, :, :])

## Improving the predictions
We can use a model with a larger number of activations to improve the results.

In [ ]:
better_ed = dl.ConvolutionalEncoderDecoder2d(
    in_channels=1,
    encoder_channels=[32, 64, 128],
    decoder_channels=[128, 64, 32],
    out_channels=1,
    # out_activation=Sigmoid, ### Carlo: already in the default
)

print(better_ed)

In [ ]:
better_regressor_template = dl.Regressor(
    model=better_ed,
    loss=L1Loss(),
    optimizer=dl.Adam(),
)
better_ed_regressor = better_regressor_template.create()
better_ed_trainer = dl.Trainer(
    max_epochs=150, accelerator="auto"
)  ### BM: 1000 epochs is a bit overkill. doesn't seem to improve much after 200 epochs
better_ed_trainer.fit(better_ed_regressor, diverse_loader)

In [ ]:
for i in range(5):
    diverse_input, diverse_target = diverse_pipeline.update().resolve()
    diverse_predicted = better_ed_regressor(diverse_input.unsqueeze(0)).detach()

    plot_image(f"Input Image {i}", diverse_input[0, :, :])
    plot_image(f"Target Image {i}", diverse_target[0, :, :])
    plot_image(f"Predicted Image {i}", diverse_predicted[0, 0, :, :])